# マックスプーリング層のクラスを実装する

In [1]:
import numpy as np
from common.util import im2col, col2im
import sys

### [演習]
* 以下のMaxPoolingクラスを完成させましょう

In [2]:
# ヒント

# 無限大
print(np.inf)

# flattenの使い方
dout =  np.random.randn(2, 3, 2, 2)
print(dout)
print()
print(dout.flatten())
print()

# 特定のインデックスの場所に代入する方法
x = np.zeros((10, 10)) 
r = np.array([0,3,5,7,9])
c = np.array([0,1,5,5,9])
x[r,c] = 1
print("x=", x)


inf
[[[[-2.38732419  0.89647349]
   [-0.0829257   0.07786273]]

  [[-0.58037442  1.43488197]
   [ 1.61098044  0.79340739]]

  [[-0.71114877  2.3355984 ]
   [-1.81699798  0.36607577]]]


 [[[ 1.85676221  0.09285084]
   [ 1.01304204  1.71180581]]

  [[ 0.5288148  -0.4353804 ]
   [-1.41261268 -0.31122252]]

  [[ 1.87747064 -1.0934979 ]
   [ 0.41732973 -1.33652561]]]]

[-2.38732419  0.89647349 -0.0829257   0.07786273 -0.58037442  1.43488197
  1.61098044  0.79340739 -0.71114877  2.3355984  -1.81699798  0.36607577
  1.85676221  0.09285084  1.01304204  1.71180581  0.5288148  -0.4353804
 -1.41261268 -0.31122252  1.87747064 -1.0934979   0.41732973 -1.33652561]

x= [[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [3]:
class MaxPooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        
        self.pool_h = pool_h # プーリングを適応する領域の高さ
        self.pool_w = pool_w # プーリングを適応する領域の幅
        self.stride = stride # ストライド数
        self.pad = pad # パディング数

        # インスタンス変数の宣言
        self.x = None
        self.arg_max = None
        self.col = None
        self.dcol = None
        
            
    def forward(self, x):
        """
        順伝播計算
        x : 入力(配列形状=(データ数, チャンネル数, 高さ, 幅))
        """        
        N, C, H, W = x.shape
        
        # 出力サイズ
        out_h = (H  + 2*self.pad - self.pool_h) // self.stride + 1 # 出力の高さ(端数は切り捨てる)
        out_w = (W + 2*self.pad - self.pool_w) // self.stride + 1# 出力の幅(端数は切り捨てる)    
        
        # プーリング演算を効率的に行えるようにするため、2次元配列に変形する
        # パディングする値は、マイナスの無限大にしておく
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad, constant_values=-np.inf)
        
        # チャンネル方向のデータが横に並んでいるので、縦に並べ替える
        # 変形後のcolの配列形状は、(N*out_h*out_w*C, pool_h*pool_w)になる 
        col = col.reshape(-1, self.pool_h*self.pool_w)

        # 最大値のインデックスを求める
        # この結果は、逆伝播計算時に用いる
        arg_max = np.argmax(col, axis=1)
        
        # 最大値を求める
        out = np.max(col, axis=1)
        
        # 画像形式に戻して、チャンネルの軸を2番目に移動する
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)

        self.x = x
        self.col = col
        self.arg_max = arg_max

        return out

    def backward(self, dout):
        """
        逆伝播計算
        マックスプーリングでは、順伝播計算時に最大値となった場所だけに勾配を伝える
        順伝播計算時に最大値となった場所は、self.arg_maxに保持されている        
        dout : 出力層側から伝わってきた勾配
        return : 入力層側へ伝える勾配
        """        
        
        # doutのチャンネル数軸を4番目に移動させる
        dout = dout.transpose(0, 2, 3, 1)
        
        # プーリング適応領域の要素数(プーリング適応領域の高さ × プーリング適応領域の幅)
        pool_size = self.pool_h * self.pool_w
        
        # 勾配を入れる配列を初期化する
        # dcolの配列形状 : (doutの全要素数, プーリング適応領域の要素数) 
        # doutの全要素数は、dout.size で取得できる
        dcol = np.zeros((dout.size, pool_size))
        
        # 順伝播計算時に最大値となった場所に、doutを配置する
        # dout.flatten()でdoutを1次元配列に変形できる
        dcol[np.arange(dcol.shape[0]), self.arg_max] = dout.flatten()
        
        # 勾配を4次元配列(データ数, チャンネル数, 高さ, 幅)に変形する
        dx = col2im(dcol, self.x.shape, self.pool_h, self.pool_w, self.stride, self.pad, is_backward=True)
        
        self.dcol = dcol # 結果を確認するために保持しておく
        
        return dx

In [6]:
# # 動作確認 条件1
# pool_h = 3
# pool_w = 3
# N = 2
# stride = 3
# pad = 0
# channel = 1
# input_size = 9

# 動作確認 条件2
pool_h = 2
pool_w = 2
N = 1
stride = 1
pad = 1
channel = 1
input_size = 2

# MaxPoolingクラスのインスタンスを生成
mp = MaxPooling(pool_h, pool_w, stride=stride, pad=pad)
print("順伝播計算")

x =  np.random.randn(N, channel, input_size, input_size)
print("x=")
print(x.round(2))
out = mp.forward(x)
print("col=")
print(mp.col.round(2))
print("out=")
print(out.round(2))
print()
print("arg_max=")
print(mp.arg_max)
print()
print()

print("逆伝播計算")
out_h = (input_size + 2*pad - pool_h) // stride + 1 # 出力の高さ
out_w =(input_size + 2*pad - pool_w) // stride + 1# 出力の幅
dout =  np.random.randn(N, channel, out_h, out_w)
print("dout=")
print(dout.round(2))
dx = mp.backward(dout)
print("dcol=")
print(mp.dcol.round(2))
print("dx=")
print(dx.round(2))
print()


順伝播計算
x=
[[[[ 0.69 -1.92]
   [ 0.14 -0.71]]]]
col=
[[ -inf  -inf  -inf  0.69]
 [ -inf  -inf  0.69 -1.92]
 [ -inf  -inf -1.92  -inf]
 [ -inf  0.69  -inf  0.14]
 [ 0.69 -1.92  0.14 -0.71]
 [-1.92  -inf -0.71  -inf]
 [ -inf  0.14  -inf  -inf]
 [ 0.14 -0.71  -inf  -inf]
 [-0.71  -inf  -inf  -inf]]
out=
[[[[ 0.69  0.69 -1.92]
   [ 0.69  0.69 -0.71]
   [ 0.14  0.14 -0.71]]]]

arg_max=
[3 2 2 1 0 2 1 0 0]


逆伝播計算
dout=
[[[[ 0.58 -0.68 -0.27]
   [ 0.31  0.89  0.61]
   [-0.11  1.22  1.69]]]]
dcol=
[[ 0.    0.    0.    0.58]
 [ 0.    0.   -0.68  0.  ]
 [ 0.    0.   -0.27  0.  ]
 [ 0.    0.31  0.    0.  ]
 [ 0.89  0.    0.    0.  ]
 [ 0.    0.    0.61  0.  ]
 [ 0.   -0.11  0.    0.  ]
 [ 1.22  0.    0.    0.  ]
 [ 1.69  0.    0.    0.  ]]
dx=
[[[[ 1.09 -0.27]
   [ 1.12  2.3 ]]]]

